<a href="https://colab.research.google.com/github/Vashi050197/Detect_Anything/blob/main/Detect_Anything.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq git+https://github.com/EduardoPach/transformers.git@adding-grounding-dino
!pip install -qq supervision

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 1.8 MB/s eta 0:00:00


In [ ]:
import cv2
import torch
import requests
import numpy as np
from PIL import Image
from tqdm import tqdm
import supervision as sv
import matplotlib.pyplot as plt
from transformers import AutoModelForObjectDetection, AutoProcessor

In [ ]:
model_id = "EduardoPacheco/grounding-dino-tiny"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForObjectDetection.from_pretrained(model_id).to(device)
processor = AutoProcessor.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def predict(images, text):
    inputs = processor(images=images, text=text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)

    # Needs to be on cpu for post_process_object_detection
    outputs.logits = outputs.logits.cpu()
    outputs.pred_boxes = outputs.pred_boxes.cpu()

    target_sizes = [image.size[::-1] for image in images]
    results = processor.image_processor.post_process_object_detection(outputs, threshold=0.35, target_sizes=target_sizes)

    frames = []
    for idx, image in enumerate(images):
        detections = sv.Detections.from_transformers(results[idx])
        annot = sv.BoundingBoxAnnotator()
        img_annot = annot.annotate(scene=np.array(image), detections=detections)
        frames.append(img_annot)

    return frames

# Video

In [ ]:
path = "ss_heel.mp4"

In [ ]:
frames = []
for frame in tqdm(sv.get_video_frames_generator(source_path=path), unit="frames"):
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    frames.append(frame_pil)

114frames [00:01, 114.40frames/s]Using bos_token, but it is not set yet.
114frames [00:11, 114.40frames/s]Using eos_token, but it is not set yet.
122frames [00:24,  5.08frames/s]


In [ ]:
# To avoid OOM error I'll predict image by image
frame_annot = []
for frame in frames:
    out = predict([frame], "a person riding a skateboard.")
    frame_annot.append(out[0])

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.


In [ ]:
def plot_images_in_grid(image_list, grid_shape=None, figsize=(10, 10)):
    """
    Plot a list of images in a grid layout.

    Parameters:
    - image_list: A list of NumPy arrays representing images.
    - grid_shape: A tuple (rows, columns) specifying the grid layout. If None, the function will attempt to create a square grid.
    - figsize: A tuple specifying the size of the entire figure.

    Example usage:
    plot_images_in_grid(image_list, grid_shape=(2, 3), figsize=(12, 8))
    """
    if grid_shape is None:
        # If grid_shape is not provided, try to create a square grid
        num_images = len(image_list)
        rows = int(np.sqrt(num_images))
        cols = int(np.ceil(num_images / rows))
    else:
        rows, cols = grid_shape

    plt.figure(figsize=figsize)

    for i, image in enumerate(image_list):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(image, cmap='gray')  # You can change the cmap as needed
        plt.axis('off')

    plt.show()

In [ ]:
def create_video_from_frames_rgb(frame_list, output_filename, fps):
    # Get the shape of the frames to determine video dimensions
    frame_height, frame_width, _ = frame_list[0].shape

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change the codec as needed
    out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

    for frame in frame_list:
        # Convert the frame from RGB to BGR
        bgr_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        # Write the frame to the video file
        out.write(bgr_frame)

    # Release the VideoWriter object
    out.release()

In [ ]:
# Example usage:
# Assuming you have a list of NumPy arrays called frame_list
# and you want to create an "output_video.mp4" at 30 frames per second (fps)
create_video_from_frames_rgb(frame_annot, "output_video.mp4", fps=30)

In [ ]:
from IPython.display import Video

# Replace 'your_video.mp4' with the actual filename of your video
video_path = 'output_video.mp4'

# Play the video
Video(video_path)